# Construct an Appropriate Initial State: Sandbox

In [1]:
from pyquil import Program
from pyquil.gates import *
from pyquil.quil import address_qubits
from pyquil.quilatom import QubitPlaceholder
from pyquil.api import QVMConnection
from pyquil.api import WavefunctionSimulator

import networkx as nx 
from toric_code import * 

Our sandbox for finding a way to correctly initialize the data qubits in our implementation of the toric code (that is, initialize the data qubits to the logical qubits state $\bar{|0\rangle}$). Not guaranteed to be free of errors or namespace conflicts. 

### Testing measurement circuits

In [2]:
# Function to measure X stabilizers from toric_code.py, repeated here for convenience 
# Not that we don't perform the measurement yet, to double-check that the circuit behaves 
# as expected 
def X_syndrome_extraction(primal_qubits: List[QubitPlaceholder]) -> Program:
    """ Runs the syndrome extraction circuit for the X-stabilizers.
    Detects phase-flip errors on the lattice.

    :param primal_qubits: List of ancilla and data qubits for the extraction.
    We assume the following input format:
            qubits = [ancilla, north, west, east, south]
    where "ancilla" is the ancilla qubit on the vertex of the primal graph,
    and "north", ... "south" are the data qubits to the north, ... south of
    the ancilla qubit. We assume the ancilla is initialized to |0>.
    :returns: Pyquil Program representing the syndrome extraction process
    """
    pq = Program()
    ro_X = pq.declare('ro', 'BIT', 1)  # Do we need to avoid namespace conflicts here?

    # Initialize the ancilla
    pq += H(primal_qubits[0])
    # Perform the circuit
    for i in range(1, len(primal_qubits)):
        pq += CNOT(primal_qubits[0], primal_qubits[i])
    # Measure in the X-basis
    # UN-COMMENT TO PERFORM THE MEASUREMENT 
    #pq += H(primal_qubits[0])
    #pq += MEASURE(primal_qubits[0], ro_X[0])

    return pq

In [3]:
# Let's verify that the above measurement returns |11111> + |00000>, assuming the data qubits 
# begin in the state |0> \tensor |0> ... \tensor |0> 
qubits = np.array([])
num_q = 5
for q in range(num_q):
    qubits = np.append(qubits, QubitPlaceholder())
print(qubits)
print(len(qubits))

[<QubitPlaceholder 43539239432> <QubitPlaceholder 43539240552>
 <QubitPlaceholder 43539240608> <QubitPlaceholder 43539240720>
 <QubitPlaceholder 43539240832>]
5


In [4]:
initial_state = X_syndrome_extraction(qubits)
initial_state = address_qubits(initial_state)
wf_sim = WavefunctionSimulator()
wavefunction = wf_sim.wavefunction(initial_state)
print(wavefunction) # Returns exactly the wavefunction we expect, if no measurement is performed 

(0.7071067812+0j)|00000> + (0.7071067812+0j)|11111>


### Constructing the initialization function

DEPRECATED in favor of ``mwpm`` algorithm (see below). 

In [5]:
def sample_without_replace(array, num_samples=2):
    """ Given an array, samples num_samples values from the array 
    and removes these values from the array. 
    """
    # Step 1: Select a random source and root node from array 
    indices = np.linspace(0, len(array) - 1, num=len(array))
    random_sample = np.random.choice(indices, size=num_samples, replace=False)
    source_and_root = array[random_sample.astype(int)] # Gives source and root nodes 
    # Step 2: Delete selected source/root nodes from array 
    reduced_array = np.delete(array, random_sample.astype(int))
    return source_and_root, reduced_array

def contains_faulty(path, faulty_nodes):
    """ Given a path as a list of nodes, and a list of faulty nodes, checks 
    whether path passes through any faulty nodes.
    """
    pass 

In [9]:
# Testing our sample w/out replacement function 
test = np.array([1, 3, 5, 7, 10])
faulty_nodes, reduced_test = sample_without_replace(test)
print(faulty_nodes)
print(reduced_test)

[ 5 10]
[1 3 7]


In [ ]:
def return_one_chain(G, faulty_nodes):
    """ Returns a list of edges corresponding to a one-chain with 
    faulty nodes on the boundary. Not guaranteed to return a valid one-chain. 
    """
    one_chain_edges = [] 
    all_faulty_nodes = faulty_nodes # Construct a copy of faulty_nodes that will not be reduced  
    while len(faulty_nodes) > 1: 
        source_root, faulty_nodes = sample_without_replace(faulty_nodes)
        path_generator = np.algorithms.simple_paths.all_simple_paths(G, source_root[0], source_root[1])
        for path in path_generator: 
            # Check that no faulty nodes exist in this path 
            path_flag = False
            if not contains_faulty(path, all_faulty_nodes): 
                # Path is valid! 
                # Convert list of nodes to list of edges 
                # path_edges = ... 
                one_chain_edges.append(path_edges)
                path_flag = True 
                break
        if not path_flag: 
            return False # We have not found a valid path between last source/root tried 
     
    if len(faulty_nodes) % 2 == 0:
        # Even number of faulty nodes, so we're done 
        return one_chain_edges
    # If the above statement doesn't execute, we have an odd number of faulty nodes; 
    # select a random non-faulty node, and find a path between last faulty node 
    # and this non-faulty node, such that the path contains none of the other faulty nodes 
    # TODO: find an elegant way to implement this. For now, we assume an even number of faulty nodes 
    return None 

## Using MWPM to construct the initial state

In [6]:
def initialize_toric_code(primal, dual, distance, L, trials=100):
    # A hacky way to ensure the initialization works: 
    # run the syndrome extraction until we return an even 
    # number of errors (okay for small graphs)
    for t in range(trials):
        # Set up programs, and reset qubits if needed 
        # Does this work? 
        primal_initial = Program() + RESET()
        dual_initial = Program() + RESET()
        # Perform syndrome extraction on primal and dual graphs 
        primal_faulty_nodes = syndrome_extraction(primal, L, primal_initial, "X")
        dual_faulty_nodes = syndrome_extraction(dual, L, dual_initial, "Z")
        test = (len(primal_faulty_nodes) % 2 == 0) and (len(dual_faulty_nodes) % 2 == 0)
        if test: 
            break
        
    assert test, "Failed to find even number of faulty nodes for primal and dual graphs"
    # With an even number of faulty nodes, we can now run the mwpm algorithm 
    # and correct the -1 eigenvalues 
    primal_correction_paths = mwpm(primal, distance, L, primal_faulty_nodes)
    dual_correction_paths = mwpm(dual, distance, L, dual_faulty_nodes)
    
    # Construct pyquil program to carry out corrections 
    primal_correct_pq = apply_operation(primal_correction_paths, primal, Z)
    dual_correct_pq = apply_operation(dual_correction_paths, dual, X)
    
    # Run the correction program 
    qvm = QVMConnection() 
    primal_correct_pq = address_qubits(primal_correct_pq)
    #dual_correct_pq = address_qubits(dual_correct_pq)
    results = qvm.run(primal_correct_pq)
    return primal_correct_pq, results

In [7]:
# Let's test the initialization function 
# NOTE that we shouldn't need to do anything to the dual graph 
# (that is, the dual graph's program will be empty), since clearly we cannot 
# introduce errors to the plaquette operators with the X-syndrome measurement 
# circuits given in our literature review 
L = 3
primal, dual, distance = construct_toric_code(L)
primal_pq, results = initialize_toric_code(primal, dual, distance, L)
print(primal_pq)

Z 0
Z 1
Z 2



In [4]:
# Does flipping a qubit on the primal graph affect the same qubit on the dual graph? 
edge_test = ((0, 0), (1, 0))
pq_test = Program()
ro = pq_test.declare('ro', 'BIT', 2)

flip_primal_qubit = primal.edges[edge_test]["data_qubit"] 
pq_test += X(flip_primal_qubit)
pq_test += MEASURE(flip_primal_qubit, ro[0])

flip_dual_qubit = dual.edges[dual_edge_to_primal_edge(edge_test, L)]["data_qubit"]
pq_test += MEASURE(flip_dual_qubit, ro[1])

In [5]:
# Run the program 
qvm = QVMConnection()
pq_test = address_qubits(pq_test)
result = qvm.run(pq_test, trials=10)
print(result) # Looks like the answer is no, but this is OKAY because we can treat the primal and dual graphs 
# completely seperately! 

[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
